# Project Setup
* This section describes how to create and configure a project

* When a user creates a project with client.create_project() the project is not ready for labeling.
    * An ontology must be set
    * datasets must be attached

In [ ]:
!pip install labelbox

In [7]:
from labelbox import Client, Project, LabelingFrontend
from labelbox.schema.ontology import Tool, OntologyBuilder
import os

In [ ]:
# Set this to a project that is already set up
PROJECT_ID = "ckm4xyfncfgja0760vpfdxoro"
# Set this if running in colab. Otherwise it should work if you have the LABELBOX_API_KEY set.
API_KEY = os.environ["LABELBOX_API_KEY"]
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

In [ ]:
client = Client(api_key = API_KEY, endpoint = ENDPOINT)

### Identify project, dataset, and ontology
* Pick the project to setup
* Dataset(s) to attach to that project
* Configure the ontology for the project

In [5]:
# Use bounding boxes to label cats
ontology_builder = OntologyBuilder(tools = [
    Tool(name = "cat", tool = Tool.Type.BBOX)
])
project = client.create_project(name = "my_new_project")
dataset = client.create_dataset(name = "my_new_dataset")
# Add data_rows since this is a new dataset (see basics to do this)

In [8]:
# Unless you are using a custom editor you should always use the following editor:
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))

In [11]:
# Note that you can use any dataset or ontology even if they already exist.
existing_project = client.get_project(PROJECT_ID)
# We are not using this, but it is possible to copy the ontology to the new project
ontology = existing_project.ontology() 

### Setup and attach dataset

In [9]:
project.setup(editor, ontology_builder.asdict())
# Could also do if ontology is a normalized ontology
# project.setup(editor, ontology.normalized)

In [10]:
project.datasets.connect(dataset)

### Review

In [16]:
# Note setup_complete will be None if it fails.
print(project.setup_complete)
print(project.ontology)
print([ds.name for ds in project.datasets()])

2021-03-17 12:35:00+00:00
['my_new_dataset']
